# Data Frame Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt


spark = SparkSession \
    .builder \
    .appName("Quiz1") \
    .getOrCreate()


path = "../sparkify_log_small.json"
user_log = spark.read.json(path)


In [3]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
user_log.filter(user_log.userId=="").select("page").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



In [14]:
unknown_id_pages = user_log.filter(user_log.userId=="").select("page").dropDuplicates().collect()

In [15]:
user_log.select("page").dropDuplicates().show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|            Home|
|       Downgrade|
|          Logout|
|   Save Settings|
|           About|
|        Settings|
|           Login|
|        NextSong|
|            Help|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



In [16]:
all_pages =user_log.select("page").dropDuplicates().collect()

In [17]:
np.setdiff1d(np.array(all_pages),np.array(unknown_id_pages))

array(['Downgrade', 'Error', 'Logout', 'NextSong', 'Save Settings',
       'Settings', 'Submit Downgrade', 'Submit Upgrade', 'Upgrade'],
      dtype='<U16')

In [18]:
(user_log.select("page").dropDuplicates().subtract(user_log.filter(user_log.userId=="").select("page").dropDuplicates())).collect()

[Row(page='Submit Downgrade'),
 Row(page='Downgrade'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='Settings'),
 Row(page='NextSong'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [20]:
user_log.filter(user_log.userId=="").select("song").dropDuplicates().show()

+----+
|song|
+----+
|null|
+----+



In [21]:
user_log.filter(user_log.userId=="").select("level").dropDuplicates().show()

+-----+
|level|
+-----+
| free|
| paid|
+-----+



In [22]:
user_log.filter(user_log.userId=="").select("status").dropDuplicates().show()

+------+
|status|
+------+
|   307|
|   200|
+------+



In [23]:
user_log.filter(user_log.userId=="").select("userAgent").dropDuplicates().show()

+---------+
|userAgent|
+---------+
|     null|
+---------+



unregistered users, or ones haven't listened to a song yet

# Question 3

How many female users do we have in the data set?

In [24]:
user_log_valid = user_log.dropna(how = "any", subset = ["userId", "sessionId"])
user_log_valid = user_log_valid.filter(user_log_valid["userId"] != "")
user_log_valid.filter(user_log_valid.gender == "F").select("userId").dropDuplicates().count()
#user_log.select("gender").show()

462

In [25]:
user_log_valid.select(["userId","gender"]).dropDuplicates().groupBy(user_log_valid.gender).count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  462|
|     M|  501|
+------+-----+



# Question 4

How many songs were played from the most played artist?

In [26]:
user_log_valid.filter(user_log_valid.page == 'NextSong').select(["artist", "song", "userId"]).groupBy("artist").count().sort("count", ascending=False).show(5)

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
+--------------------+-----+
only showing top 5 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [39]:
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

In [43]:
cusum = user_log_valid.filter((user_log_valid.page == 'NextSong') | (user_log_valid.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

In [45]:
cusum.show(15)

+------+--------+-------------+---------+------+
|userID|    page|           ts|homevisit|period|
+------+--------+-------------+---------+------+
|  1436|NextSong|1513783259284|        0|     0|
|  1436|NextSong|1513782858284|        0|     0|
|  2088|    Home|1513805972284|        1|     1|
|  2088|NextSong|1513805859284|        0|     1|
|  2088|NextSong|1513805494284|        0|     1|
|  2088|NextSong|1513805065284|        0|     1|
|  2088|NextSong|1513804786284|        0|     1|
|  2088|NextSong|1513804555284|        0|     1|
|  2088|NextSong|1513804196284|        0|     1|
|  2088|NextSong|1513803967284|        0|     1|
|  2088|NextSong|1513803820284|        0|     1|
|  2088|NextSong|1513803651284|        0|     1|
|  2088|NextSong|1513803413284|        0|     1|
|  2088|NextSong|1513803254284|        0|     1|
|  2088|NextSong|1513803057284|        0|     1|
+------+--------+-------------+---------+------+
only showing top 15 rows



In [46]:
cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

